# Advertising spotting

### Requirements

- Pandas

In [1]:
import pandas as pd
import numpy as np 

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin

import matplotlib.pyplot as plt

from prepare_data import get_train_test_public

# Data exploration

In [2]:
X_train, X_test, y_train, y_test = get_train_test_public()

In [3]:
X_train.head()

,shot_length,motion_distribution_mean,motion_distribution_variance,frame_difference_distribution_mean,frame_difference_distribution_variance,short_time_energy_mean,short_time_energy_variance,ZCR_mean,ZCR_variance,spectral_centroid_mean,...,bag_of_audio_words_3994,bag_of_audio_words_3995,bag_of_audio_words_3996,bag_of_audio_words_3997,bag_of_audio_words_3998,bag_of_audio_words_3999,bag_of_audio_words_4000,edge_change_ratio_mean,edge_change_ratio_variance,channel
0,39.0,0.358888,0.179183,2.207158,1.448997,0.011593,0.010000,0.135016,0.092428,3539.445312,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.413140,0.485093,CNN
1,354.0,1.261189,0.785149,5.846283,5.492986,0.018532,0.011248,0.126942,0.069577,3912.496582,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.453395,0.277466,CNN
2,39.0,0.977077,1.535457,7.017412,10.836514,0.017693,0.010044,0.122276,0.054648,3576.808838,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.709121,0.679148,CNNIBN
3,164.0,2.187399,2.278633,13.425022,16.933500,0.015677,0.009293,0.083365,0.047887,3843.954834,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.201829,0.504669,CNNIBN
4,25.0,6.117249,5.267529,26.422398,14.661750,0.013660,0.006832,0.070750,0.046628,3334.407227,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.581734,0.411938,CNNIBN


In [4]:
y_train.unique()

array([ 1, -1], dtype=int64)

In [5]:
y_train[y_train==-1]=0
y_train.unique()

array([1, 0], dtype=int64)

# Simple model

In [7]:
class Scaler(BaseEstimator, TransformerMixin):
    def __init__(self, scaler):
        self.scaler = scaler

    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self

    def transform(self, X_df):
        idx = X_df.index
        col = X_df.columns
        res = self.scaler.transform(X_df)
        res = pd.DataFrame(res,index=idx,columns=col)

        return res
  
scaling = Scaler(StandardScaler())
logreg = LogisticRegression(max_iter=100)

pipe = make_pipeline(scaling, logreg)

In [8]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)